<a href="https://colab.research.google.com/github/nyanta012/demo/blob/main/rinna_4b_minigtp4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!git clone https://github.com/Vision-CAIR/MiniGPT-4.git
%cd ./MiniGPT-4
!git checkout 22d8888 # latest version as of July 31, 2023.
!wget https://huggingface.co/rinna/bilingual-gpt-neox-4b-minigpt4/resolve/main/customized_mini_gpt4.py
!wget https://huggingface.co/rinna/bilingual-gpt-neox-4b-minigpt4/resolve/main/checkpoint.pth

!pip install accelerate==0.16.0 aiohttp==3.8.4 aiosignal==1.3.1 async-timeout==4.0.2 attrs==22.2.0 bitsandbytes==0.37.0 cchardet==2.1.7 chardet==5.1.0 contourpy==1.0.7 cycler==0.11.0 filelock==3.9.0 fonttools==4.38.0 frozenlist==1.3.3 huggingface-hub==0.13.4 importlib-resources==5.12.0 kiwisolver==1.4.4 matplotlib==3.7.0 multidict==6.0.4 openai==0.27.0 packaging==23.0 psutil==5.9.4 pycocotools==2.0.6 pyparsing==3.0.9 python-dateutil==2.8.2 pyyaml==6.0 regex==2022.10.31 tokenizers==0.13.2 tqdm==4.64.1 transformers==4.28.0 timm==0.6.13 spacy==3.5.1 webdataset==0.2.48 scikit-learn==1.2.2 scipy==1.10.1 yarl==1.8.2 zipp==3.14.0 omegaconf==2.3.0 opencv-python==4.7.0.72 iopath==0.1.10 decord==0.6.0 tenacity==8.2.2 peft pycocoevalcap sentence-transformers umap-learn notebook gradio==3.24.1 gradio-client==0.0.8 wandb

In [2]:
import torch
import requests
from PIL import Image
from minigpt4.processors.blip_processors import Blip2ImageEvalProcessor
from customized_mini_gpt4 import CustomizedMiniGPT4

ckpt_path = "./checkpoint.pth"

model = CustomizedMiniGPT4(gpt_neox_model="rinna/bilingual-gpt-neox-4b")
tokenizer = model.gpt_neox_tokenizer

if torch.cuda.is_available():
    model = model.to("cuda")

if ckpt_path is not None:
    print("Load BLIP2-LLM Checkpoint: {}".format(ckpt_path))
    ckpt = torch.load(ckpt_path, map_location="cpu")
    model.load_state_dict(ckpt['model'], strict=False)

/usr/local/lib/python3.10/dist-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


Loading VIT


100%|██████████| 1.89G/1.89G [01:05<00:00, 30.8MB/s]


freeze vision encoder
Loading VIT Done
Loading Q-Former


100%|██████████| 413M/413M [00:15<00:00, 28.2MB/s]


freeze Qformer
Loading Q-Former Done
Loading LLM


Loading LLM Done
Load BLIP2-LLM Checkpoint: ./checkpoint.pth


In [10]:
prompt = [
    {
        "speaker": "ユーザー",
        "text": "<Img><ImageHere></Img> 猫は何を見ていますか？"
    },
]
prompt = [
    f"{uttr['speaker']}: {uttr['text']}"
    for uttr in prompt
]
prompt = "\n".join(prompt)
prompt = (
    prompt
    + "\n"
    + "システム: "
)

In [11]:
from PIL import Image

image_path = "/content/kotowaza_neko_koban.png"
raw_image = Image.open(image_path).convert('RGB')

In [12]:
vis_processor = Blip2ImageEvalProcessor()

image = vis_processor(raw_image).unsqueeze(0).to(model.device)
image_emb = model.encode_img(image)

embs = model.get_context_emb(prompt, [image_emb])

output_ids = model.gpt_neox_model.generate(
    inputs_embeds=embs,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.1,
    top_p=0.85,
    pad_token_id=tokenizer.pad_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id
)

output = tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True)
print(output)


お腹を触っています。
